# Tutorial 11: Working with Structured Data

In this tutorial, we'll explore how to work with structured data in LangChain and LangGraph applications. We'll use Pydantic for data modeling, create structured inputs and outputs, and leverage the JSON Toolkit for complex data manipulation.

## Setup

First, let's import the necessary libraries and set up our environment:

In [ ]:
import os
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.llms import Groq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain.tools import BaseTool, StructuredTool, Tool
from langchain.agents import AgentExecutor, Tool
from langchain.agents.structured_chat.base import StructuredChatAgent
from langgraph.graph import StateGraph, END

# Set up the Groq LLM
llm = Groq(api_key=os.environ["GROQ_API_KEY"])

## 1. Introduction to Pydantic for data modeling

Pydantic is a powerful library for data validation and settings management using Python type annotations. Let's start by creating a simple Pydantic model:

In [ ]:
class Person(BaseModel):
    name: str
    age: int
    email: Optional[str] = None

# Create a Person instance
alice = Person(name="Alice", age=30, email="alice@example.com")
print(alice)

# Pydantic will raise a validation error if the data doesn't match the model
try:
    invalid_person = Person(name="Bob", age="thirty")
except ValueError as e:
    print(f"Validation error: {e}")

## 2. Creating structured inputs and outputs with Pydantic

Now, let's use Pydantic with LangChain to create structured outputs from our LLM:

In [ ]:
class MovieReview(BaseModel):
    title: str = Field(description="The title of the movie")
    year: int = Field(description="The year the movie was released")
    rating: float = Field(description="The rating of the movie on a scale of 0 to 10")
    summary: str = Field(description="A brief summary of the movie's plot")

movie_review_parser = PydanticOutputParser(pydantic_object=MovieReview)

movie_review_prompt = ChatPromptTemplate.from_template(
    """Provide a brief review for the movie {movie_title}. 
    {format_instructions}
    """
)

movie_review_chain = LLMChain(
    llm=llm,
    prompt=movie_review_prompt,
    output_parser=movie_review_parser
)

result = movie_review_chain.run(
    movie_title="The Matrix",
    format_instructions=movie_review_parser.get_format_instructions()
)

print(result)

## 3. Using the JSON Toolkit for complex data manipulation

LangChain provides a JSON Toolkit for working with complex JSON data. Let's create a simple example using this toolkit:

In [ ]:
import json
from langchain.tools.json.tool import JsonSpec
from langchain.agents import create_json_agent
from langchain.agents.agent_toolkits import JsonToolkit

# Sample JSON data
json_data = {
    "movies": [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010},
        {"title": "The Shawshank Redemption", "director": "Frank Darabont", "year": 1994},
        {"title": "The Godfather", "director": "Francis Ford Coppola", "year": 1972}
    ]
}

json_spec = JsonSpec(dict_=json_data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=llm,
    toolkit=json_toolkit,
    verbose=True
)

result = json_agent_executor.run("What is the oldest movie in the list?")
print(result)

## 4. Integrating structured data with LangChain and LangGraph

Now, let's create a more complex example that combines structured data with LangChain and LangGraph. We'll build a movie recommendation system:

In [ ]:
class Movie(BaseModel):
    title: str
    genre: str
    year: int
    director: str

class MovieRecommendation(BaseModel):
    recommended_movie: Movie
    reason: str

class MovieDatabase:
    def __init__(self):
        self.movies = [
            Movie(title="The Shawshank Redemption", genre="Drama", year=1994, director="Frank Darabont"),
            Movie(title="The Godfather", genre="Crime", year=1972, director="Francis Ford Coppola"),
            Movie(title="Pulp Fiction", genre="Crime", year=1994, director="Quentin Tarantino"),
            Movie(title="The Dark Knight", genre="Action", year=2008, director="Christopher Nolan"),
            Movie(title="Forrest Gump", genre="Drama", year=1994, director="Robert Zemeckis")
        ]

    def get_movies(self):
        return [movie.dict() for movie in self.movies]

movie_db = MovieDatabase()

def get_movie_recommendation(preferences: str) -> MovieRecommendation:
    movies = movie_db.get_movies()
    prompt = ChatPromptTemplate.from_template(
        """Based on the user's preferences: '{preferences}', recommend a movie from the following list:
        {movies}
        
        Provide your recommendation in the following format:
        {format_instructions}
        """
    )
    
    parser = PydanticOutputParser(pydantic_object=MovieRecommendation)
    
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        output_parser=parser
    )
    
    result = chain.run(
        preferences=preferences,
        movies=json.dumps(movies, indent=2),
        format_instructions=parser.get_format_instructions()
    )
    
    return result

recommendation_tool = StructuredTool.from_function(
    func=get_movie_recommendation,
    name="MovieRecommendation",
    description="Recommends a movie based on user preferences"
)

tools = [recommendation_tool]

agent = StructuredChatAgent.from_llm_and_tools(llm=llm, tools=tools)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)

result = agent_executor.run("I'm in the mood for a classic crime movie. What do you recommend?")
print(result)

## Conclusion

In this tutorial, we've explored how to work with structured data in LangChain and LangGraph applications. We've covered:

1. Using Pydantic for data modeling and validation
2. Creating structured inputs and outputs with Pydantic and LangChain
3. Using the JSON Toolkit for complex data manipulation
4. Integrating structured data with LangChain and LangGraph in a movie recommendation system

These techniques allow you to create more robust and type-safe applications, making it easier to work with complex data structures in your AI-powered systems.

## Next Steps

To further improve your skills in working with structured data in LangChain and LangGraph applications, consider the following:

1. Experiment with more complex Pydantic models and nested structures
2. Explore other LangChain tools and agents that work with structured data
3. Implement error handling and fallback strategies for parsing failures
4. Integrate external APIs or databases to work with real-world data
5. Develop a full-fledged application that combines multiple structured data techniques